# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [52]:
import math
import numpy as np
import pandas as pd
import xlsxwriter
from scipy import stats
import yfinance as yf

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,A,Agilent Technologies,Health Care,Life Sciences Tools & Services,"Santa Clara, California",05/06/00,1090872,1999
1,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",30/11/82,320193,1977
2,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",31/12/12,1551152,2013 (1888)
3,ABNB,Airbnb,Consumer Discretionary,"Hotels, Resorts & Cruise Lines","San Francisco, California",18/09/23,1559720,2008
4,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",04/03/57,1800,1888
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",01/11/11,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",06/10/97,1041061,1997
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",07/08/01,1136869,1927
501,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",23/12/19,877212,1969


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [4]:
symbol = 'AAPL'

data = yf.Ticker(symbol).info
data

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and p

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [5]:
price = data['currentPrice']
trailing_pe = data['trailingPE']
trailing_pe

37.602627

## Executing An API Call For All Stocks & Building Our DataFrame

Just like in our first project, it's now time to execute API calls and add the information we need to our DataFrame.


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [9]:
my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns=my_columns)

for ticker in stocks['Symbol']:
    data = yf.Ticker(ticker).info
    try:
        pe_ratio = data['trailingPE']
    except KeyError:
        pe_ratio = data['forwardPE']
    
    new_row = pd.DataFrame([[ticker, data['currentPrice'], pe_ratio, 'N/A']], columns=my_columns)

    final_df = pd.concat([final_df, new_row], ignore_index=True)
final_df    

/var/folders/hd/h4ty4f0j2tgfss4vb7rrr93w0000gn/T/ipykernel_3038/1545358673.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, new_row], ignore_index=True)


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,128.57,26.729733,N/A
1,AAPL,229.00,37.602627,N/A
2,ABBV,167.76,58.45296,N/A
3,ABNB,135.25,46.961803,N/A
4,ABT,115.93,35.237083,N/A
...,...,...,...,...
498,XYL,122.82,35.293102,N/A
499,YUM,132.88,24.791044,N/A
500,ZBH,109.27,20.853054,N/A
501,ZBRA,384.53,52.388283,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [10]:
final_df['Price-to-Earnings Ratio'] = final_df['Price-to-Earnings Ratio'].astype(float)
final_df.sort_values('Price-to-Earnings Ratio', inplace=True)
final_df = final_df[final_df['Price-to-Earnings Ratio'] > 0]
final_df = final_df[:50]
final_df.reset_index(inplace=True)
final_df.drop('index', axis=1, inplace=True)
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,APA,22.13,3.143466,N/A
1,FMC,57.20,4.692371,N/A
2,VTRS,13.13,4.944231,N/A
3,WBA,8.24,5.213111,N/A
4,GM,54.87,5.855923,N/A
5,APTV,52.59,5.856348,N/A
6,EG,376.87,5.878490,N/A
7,ACGL,95.71,6.423490,N/A
8,PARA,10.58,6.514577,N/A
9,DVN,38.30,7.092592,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [11]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [12]:
portfolio_input()

That's not a number! 
 Try again:


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [17]:
position_size = float(portfolio_size) / len(final_df.index)

for row in final_df.index:
    final_df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_df.loc[row, 'Price'])
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,APA,22.13,3.143466,903
1,FMC,57.20,4.692371,349
2,VTRS,13.13,4.944231,1523
3,WBA,8.24,5.213111,2427
4,GM,54.87,5.855923,364
5,APTV,52.59,5.856348,380
6,EG,376.87,5.878490,53
7,ACGL,95.71,6.423490,208
8,PARA,10.58,6.514577,1890
9,DVN,38.30,7.092592,522


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [62]:
writer = pd.ExcelWriter('value_strat.xlsx', engine='xlsxwriter')
final_df.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [63]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [64]:
col_formats = {
                'A' : ['Ticker', string_template],
                'B' : ['Price', dollar_template],
                'C' : ['Price-to-Earnings Ratio', float_template],
                'D' : ['Number of Shares to Buy', integer_template]
              }

for column in col_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, col_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', col_formats[column][0], col_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [65]:
writer.close()